# import library

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
okt = Okt()
kkma=Kkma()
mecab=Mecab(dicpath=r"C:\mecab\mecab-ko-dic") 
komoran=Komoran()
hannanum=Hannanum()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# data load

In [ ]:
data11_from_277541_to_37=pd.read_csv('./data11_from_277541_to_37.csv', )
data11_from_520037_to_277264=pd.read_csv('./data11_from_520037_to_277264.csv')
data12_from_135461_to_62444=pd.read_csv('./data12_from_135461_to_62444.csv')
data12_from_317154_to_135519=pd.read_csv('./data12_from_317154_to_135519.csv')
data12_from_456597_to_317136=pd.read_csv('./data12_from_456597_to_317136.csv')
data12_from_520015_to_456369=pd.read_csv('./data12_from_520015_to_456369.csv')
data12_from_62438_to_38=pd.read_csv('./data12_from_62438_to_38.csv')
data13_from_300811_to_39=pd.read_csv('./data13_from_300811_to_39.csv')
data13_from_519990_to_301071=pd.read_csv('./data13_from_519990_to_301071.csv')

In [ ]:
mbtidata_101_128_72=pd.read_csv('./mbtidata_101_128_72.csv')
data28=pd.read_csv('./data28_from_520142_to_273525.csv')

In [ ]:
data16_from_520598_to_42=pd.read_csv('./data16_from_520598_to_42.csv')
data16_from_520598_to_519889=pd.read_csv('./data16_from_520598_to_519889.csv')
data16=pd.concat([data16_from_520598_to_42,data16_from_520598_to_519889], axis=0)
data14=pd.read_csv('./data14_from_520893_to_40.csv')
data15=pd.read_csv('./data15_from_520701_to_41.csv')

In [ ]:
data18_from_244601_to_35561=pd.read_csv('./data18_from_244601_to_35561.csv')
data18_from_418760_to_244386=pd.read_csv('./data18_from_418760_to_244386.csv')
data18_from_520161_to_419044=pd.read_csv('./data18_from_520161_to_419044.csv')

data17_from_102143_to_1915=pd.read_csv('./data17_from_102143_to_1915.csv')
data17_from_333281_to_101367=pd.read_csv('./data17_from_333281_to_101367.csv')
data17_from_390309_to_333215=pd.read_csv('./data17_from_390309_to_333215.csv')
data17_from_520127_to_391024=pd.read_csv('./data17_from_520127_to_391024.csv')

data18=pd.concat([data18_from_244601_to_35561,data18_from_418760_to_244386,data18_from_520161_to_419044], axis=0)
data17=pd.concat([data17_from_102143_to_1915,data17_from_333281_to_101367,data17_from_390309_to_333215,data17_from_520127_to_391024], axis=0)

In [ ]:
data11=pd.concat([data11_from_277541_to_37,data11_from_520037_to_277264], axis=0)
data12=pd.concat([data12_from_135461_to_62444,data12_from_317154_to_135519,data12_from_456597_to_317136,data12_from_520015_to_456369,data12_from_62438_to_38], axis=0)
data13=pd.concat([data13_from_300811_to_39,data13_from_519990_to_301071], axis=0)

In [ ]:
data11['menu_id']=11
data12['menu_id']=12
data13['menu_id']=13
data28['menu_id']=28
data14['menu_id']=14
data15['menu_id']=15
data16['menu_id']=16
data17['menu_id']=17
data18['menu_id']=18

In [ ]:
data=pd.concat([data11,data12,data13,mbtidata_101_128_72, data28, data14, data15,data16,data17,data18], axis=0)

In [ ]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.mbti=data.mbti.apply(lambda x : np.nan if x.find('ui')!=-1 else x)
data.dropna(inplace=True)

In [ ]:
data=data.reset_index().drop(['index'], axis=1)

In [ ]:
data

# preprocessing

## 전처리 함수 정의

In [ ]:
# 전처리 함수 정의
def cleansing(text):  

    text=text[2:-2]
    
    pattern = '([u200b]+)'         
    text = re.sub(pattern=pattern, repl=' ', string=text)
    
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\.[a-zA-Z0-9-.]+)' # e-mail 주소 제거  
    text = re.sub(pattern=pattern,repl=' ',string=text)
    
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{3}))+'  # url 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)
    
    pattern = 'www.(?:[-\w.]|(?:\da-fA-F]{2}))+'  # url 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)  
    
    pattern = r'\([^)]*\)' # () 속 문자 제거
    text = re.sub(pattern=pattern, repl=' ', string=str(text)) 
    
    pattern = r'\[[^]]*\]' # [] 속 문자 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    
    pattern = '\d\W'                                          # 숫자+ 특수분자 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)
    
    pattern = '[\r|\n]'                                          # \r, \n 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)

    pattern = '[^\w\s\.\,\%\~]'                                          # 특수기호 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)
    
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)

    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    
    pattern = '인프피|인픞' 
    text = re.sub(pattern=pattern,repl='infp',string=text)
    
    pattern = '엔프피|엔픞' 
    text = re.sub(pattern=pattern,repl='enfp',string=text)
    
    pattern = '인팁' 
    text = re.sub(pattern=pattern,repl='intp',string=text)
    
    pattern = '엔팁' # 엔 팁
    text = re.sub(pattern=pattern,repl='entp',string=text)
    
    pattern = '잇프피|잇픞' 
    text = re.sub(pattern=pattern,repl='isfp',string=text)
    
    pattern = '엣프피|엣픞' 
    text = re.sub(pattern=pattern,repl='esfp',string=text)
    
    pattern = '잇팁' 
    text = re.sub(pattern=pattern,repl='istp',string=text)
    
    pattern = '엣팁' 
    text = re.sub(pattern=pattern,repl='estp',string=text)
    
    pattern = '인프제' 
    text = re.sub(pattern=pattern,repl='infj',string=text)
    
    pattern = '엔프제' 
    text = re.sub(pattern=pattern,repl='enfj',string=text)
    
    pattern = '잇프제' 
    text = re.sub(pattern=pattern,repl='isfj',string=text)
    
    pattern = '엣프제' 
    text = re.sub(pattern=pattern,repl='esfj',string=text)
    
    pattern = '인티제' 
    text = re.sub(pattern=pattern,repl='intj',string=text)
    
    pattern = '엔티제' 
    text = re.sub(pattern=pattern,repl='entj',string=text)
    
    pattern = '잇티제|이티제' # 이티제
    text = re.sub(pattern=pattern,repl='istj',string=text)
    
    pattern = '엣티제|에스티제' # 에스티제
    text = re.sub(pattern=pattern,repl='estj',string=text)
    
    pattern = 'enfj|enfp|entj|entp|esfj|esfp|estj|estp|infj|infp|intj|intp|isfj|isfp|istj|istp'         
    text = re.sub(pattern=pattern, repl=' ', string=text)
    
    pattern = r'[\\n]+'    
    text = re.sub(pattern=pattern, repl='', string=text)

    pattern = re.compile(r'\s+')                                 # 이중 space 제거
    text = re.sub(pattern=pattern,repl=' ',string=text)
    
    return(text)

In [ ]:
data1=data.copy()
for i in tqdm(range(len(data1))):
    data1.loc[i,'content']=cleansing(data1.loc[i,'content']) # 정규표현식
#     data1.loc[i,'content']=okt.normalize(data1.loc[i,'content']) # 다듬기

In [ ]:
# content 길이 plot
length=data1['content'].astype(str).apply(len)
plt.hist(length, bins=50, alpha=0.5, color='r', label='word')
plt.title('histogram of length of content')
plt.figure(figsize=(12, 5))
plt.boxplot(length, labels=['counts'], showmeans=True)
print('content 길이 최댓값: {}'.format(np.max(length)))
print('content 길이 최솟값: {}'.format(np.min(length)))
print('content 길이 평균값: {}'.format(np.mean(length)))
print('content 길이 중간값: {}'.format(np.median(length)))

In [ ]:
# content 길이 10퍼센트 미만인 글 제거
data1.drop(data1[length<np.percentile(length,10)].index, inplace=True)
data1=data1.reset_index().drop(['index'], axis=1)

In [ ]:
data2=data1.copy()

In [ ]:
def replace_mbti(text):
    pattern = '인프피|인픞' 
    text = re.sub(pattern=pattern,repl='INFP',string=text)
    
    pattern = '엔프피|엔픞' 
    text = re.sub(pattern=pattern,repl='ENFP',string=text)
    
    pattern = '인팁' 
    text = re.sub(pattern=pattern,repl='INTP',string=text)
    
    pattern = '엔팁' # 엔 팁
    text = re.sub(pattern=pattern,repl='ENTP',string=text)
    
    pattern = '잇프피|잇픞' 
    text = re.sub(pattern=pattern,repl='ISFP',string=text)
    
    pattern = '엣프피|엣픞' 
    text = re.sub(pattern=pattern,repl='ESFP',string=text)
    
    pattern = '잇팁' 
    text = re.sub(pattern=pattern,repl='ISTP',string=text)
    
    pattern = '엣팁' 
    text = re.sub(pattern=pattern,repl='ESTP',string=text)
    
    pattern = '인프제' 
    text = re.sub(pattern=pattern,repl='INFJ',string=text)
    
    pattern = '엔프제' 
    text = re.sub(pattern=pattern,repl='ENFJ',string=text)
    
    pattern = '잇프제' 
    text = re.sub(pattern=pattern,repl='ISFJ',string=text)
    
    pattern = '엣프제' 
    text = re.sub(pattern=pattern,repl='ESFJ',string=text)
    
    pattern = '인티제' 
    text = re.sub(pattern=pattern,repl='INTJ',string=text)
    
    pattern = '엔티제' 
    text = re.sub(pattern=pattern,repl='ENTJ',string=text)
    
    pattern = '잇티제|이티제' # 이티제
    text = re.sub(pattern=pattern,repl='ISTJ',string=text)
    
    pattern = '엣티제|에스티제' # 에스티제
    text = re.sub(pattern=pattern,repl='ESTJ',string=text)
    
    return (text)

In [ ]:
for i in tqdm(range(len(data2))):
    data2.loc[i,'mbti']=replace_mbti(data2.loc[i,'mbti'])
    data2.loc[i,'mbti']=data2.loc[i,'mbti'].lower()

## labeling

In [ ]:
# labeling 16개 (0-15)
encoder=LabelEncoder()
data2['mbti_label']=encoder.fit_transform(data2['mbti'])

In [ ]:
# map (0-15)
encoder.classes_

In [ ]:
# labeling binary
e=[0,1,2,3,4,5,6,7] 
n=[0,1,2,3,8,9,10,11] 
f=[0,1,4,5,8,9,12,13]
j=[0,2,4,6,8,10,12,14]

# 앞의 알파벳이 1
data2['e_i']=data2['mbti_label'].apply(lambda x: 1 if x in e else 0) 
data2['n_s']=data2['mbti_label'].apply(lambda x: 1 if x in n else 0) 
data2['f_t']=data2['mbti_label'].apply(lambda x: 1 if x in f else 0) 
data2['j_p']=data2['mbti_label'].apply(lambda x: 1 if x in j else 0) 

In [ ]:
data2=data2.reset_index().drop(['index'], axis=1)

In [ ]:
data2.info()

## 텍스트 정규화

In [ ]:
# 한국어 띄어쓰기 (안깔림)
# from pykospacing import Spacing

In [ ]:
# # 한국어 맞춤법 검사
# from hanspell import spell_checker


# for i in tqdm(range(len(data2))):
#     data2.loc[i,'content']=spell_checker.check(data2.loc[i,'content']).checked 

In [ ]:
# # 한국어 반복되는 문자 정제
# from soynlp.normalizer import *

# for i in tqdm(range(len(data2))):
#     data2.loc[i,'content']=repeat_normalize(data2.loc[i,'content'], num_repeats=2)


In [ ]:
# # 데이터 확인
# i=18
# print(data1.content.loc[i,]+'\n')
# print(data2.content.loc[i,])

In [ ]:
length=data2['content'].astype(str).apply(len)
# content 길이 0인 글 제거
data2.drop(data2[length==0].index, inplace=True)

In [ ]:
data2=data2.reset_index().drop(['index'], axis=1)

# EDA

In [ ]:
sns.countplot(x='mbti', data=data2)

In [ ]:
sns.countplot(x='e_i', data=data2)
print(sum(data2.e_i==1)/len(data2))

In [ ]:
sns.countplot(x='n_s', data=data2)
sum(data2.n_s==1)/len(data2)

In [ ]:
sns.countplot(x='f_t', data=data2)
sum(data2.f_t==1)/len(data2)

In [ ]:
sns.countplot(x='j_p', data=data2)
sum(data2.j_p==1)/len(data2)

In [ ]:
# 전처리 후 남은 데이터 percentage
data2.shape[0]/data.shape[0]*100